## TensorFlow on turbotec

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
import tensorflow as tf
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
from scipy.io import loadmat
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
cd /content/gdrive/My\ Drive/Vahid\ Ghafourian

/content/gdrive/My Drive/Vahid Ghafourian


### Load data

In [10]:
# Get temperature data and make it as a list in order to have it as a feature in dataset
dst = loadmat('../datasets/data_trend_tem.mat')
dst_l = dst['trend_tem'].tolist()[0]
dst_list = np.asarray(dst_l)

In [11]:
# Load all mat data files in a list format
dataDir = "../datasets/"
mats = []
for file in os.listdir(dataDir):
     if file.endswith(".mat"):
        mats.append(loadmat(dataDir+file))

### Create DataFrames From mat Files

In [12]:
# Convert mat data files to pandas dataframes and give names to the columns

cols = ['Power','Input DG','Output Temp','Output Press','Fuel Debi','TIT','TET']
df = {}

for i in range(len(mats)-1):
    df[i] = pd.DataFrame(list(mats[i].values())[3],columns= cols)
    df[i]['Temperature'] = dst_list
    if i == 0:
        df[i]['State'] = '0'
    elif i == 1:
        df[i]['State'] = '1'
    elif i == 2:
        df[i]['State'] = '2'
    elif i == 3:
        df[i]['State'] = '3'
    elif i == 4:
        df[i]['State'] = '4'
    elif i == 5:
        df[i]['State'] = '5'
    elif i == 6:
        df[i]['State'] = '6'
    elif i == 7:
        df[i]['State'] = '7'
    elif i == 8:
        df[i]['State'] = '8'
    elif i == 9:
        df[i]['State'] = '9'
    elif i == 10:
        df[i]['State'] = '10'
    elif i == 11:
        df[i]['State'] = '11'
    elif i == 12:
        df[i]['State'] = '12'
    elif i == 13:
        df[i]['State'] = '13' 

### Select Samples and Merge All Samples into One DataFrame and Shuffle the Samples

In [13]:
# Select Random Samples From all data for training
sample = {}
for i in range(len(mats)-1):
    if i == 0:
        sample[i] = df[i].sample(n= 600, replace = False, random_state = 0)
    else:    
        sample[i] = df[i].sample(n= 80, replace = False, random_state = 0)
total_sample = sample[0].append([sample[1],sample[2], sample[3],sample[4], sample[5],sample[6], sample[7],
                                 sample[8],sample[9],sample[10],sample[11],sample[12],sample[13]],
                                ignore_index=True)

# Shuffle the total sample data
total_sample = total_sample.sample(frac=1)

In [14]:
# Set X and y of the total sample data
X = total_sample[['Power','Input DG','Output Temp','Output Press','Fuel Debi','TIT','TET','Temperature']]
# X = total_sample.iloc[:, 0:8]
y = total_sample['State']

### Create Train and Validation Sets

In [15]:
# split total sample data into test, train and validation sets

# create train set and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7, random_state=0)

# print train and validation sets proportions
print('train: {}% | validation {}%'.format(round(len(y_train)/len(y),2)*100,round(len(y_val)/len(y),2)*100))
print('train samples: {} | validation samples: {}' .format(X_train.shape[0],X_val.shape[0]))

train: 70.0% | validation 30.0%
train samples: 1148 | validation samples: 492


In [16]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu',input_shape=X_train.shape),
    tf.keras.layers.Dense(400, activation='relu'),
    tf.keras.layers.Dense(400, activation='relu'),
    tf.keras.layers.Dense(14, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1148, 8)           72        
_________________________________________________________________
dense_1 (Dense)              (None, 1148, 400)         3600      
_________________________________________________________________
dense_2 (Dense)              (None, 1148, 400)         160400    
_________________________________________________________________
dense_3 (Dense)              (None, 1148, 14)          5614      
Total params: 169,686
Trainable params: 169,686
Non-trainable params: 0
_________________________________________________________________


In [18]:
tf.config.experimental_run_functions_eagerly(True)
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
36/36 [==============================] - 1s 19ms/step - loss: 2.4996 - accuracy: 0.3415
Epoch 2/10
36/36 [==============================] - 1s 20ms/step - loss: 2.4068 - accuracy: 0.3563
Epoch 3/10
36/36 [==============================] - 1s 20ms/step - loss: 2.3880 - accuracy: 0.3563
Epoch 4/10
36/36 [==============================] - 1s 20ms/step - loss: 2.3630 - accuracy: 0.3563
Epoch 5/10
36/36 [==============================] - 1s 20ms/step - loss: 2.3520 - accuracy: 0.3563
Epoch 6/10
36/36 [==============================] - 1s 20ms/step - loss: 2.3153 - accuracy: 0.3563
Epoch 7/10
36/36 [==============================] - 1s 19ms/step - loss: 2.3288 - accuracy: 0.3554
Epoch 8/10
36/36 [==============================] - 1s 19ms/step - loss: 2.3088 - accuracy: 0.3563
Epoch 9/10
36/36 [==============================] - 1s 18ms/step - loss: 2.2973 - accuracy: 0.3563
Epoch 10/10
36/36 [==============================] - 1s 19ms/step - loss: 2.2977 - accuracy: 0.3563


In [20]:
model.evaluate(X_val,  y_val)

16/16 [==============================] - 0s 11ms/step - loss: 2.2390 - accuracy: 0.3882


[2.2389867305755615, 0.38821136951446533]